In [2]:
import pandas as pd
import numpy as np 
from dataCleaning import read_run, column_clean, preprocessing, create_sensor_col
import pdb

In [17]:
def read_run(filename, skiprows=7): #skip the first 7 rows (freq/cycle time fields as well as metadata)
    usecols = list(range(0, 56)) 
    df = pd.read_csv(filename, low_memory = False, 
                     header = 0,  
                     skiprows=skiprows,
                    #  names=header,
                     usecols = usecols,
                     on_bad_lines='skip') 
    df.columns = ['RDelt_EMG_TimeSeries', 'RDelt_EMG_MilliVolts', 'RDelt_IMU_Acc X Time Series(s)', 'RDelt_ACC X (G)', 'RDelt_Acc Y Time Series(s)',   'RDelt_ACC Y (G)',  'RDelt_Acc Z Time Series(s)', 'RDelt_ACC Z (G)','RDelt_GyroXTime Series(s)', 'RDelt_GYRO X (deg/s)','RDelt_GyroYTime Series(s)', 'RDelt_GYRO Y (deg/s)', 'RDelt_GyroZTime Series(s)', 'RDelt_GYRO Z (deg/s)',
                 'LDelt_TimeSeries', 'LDelt_MilliVolts', 'LDelt_Acc X Time Series(s)', 'LDelt_ACC X (G)', 'LDelt_Acc Y Time Series(s)',   'LDelt_ACC Y (G)',  'LDelt_Acc Z Time Series(s)', 'LDelt_ACC Z (G)','LDelt_GyroXTime Series(s)', 'LDelt_GYRO X (deg/s)','LDelt_GyroYTime Series(s)', 'LDelt_GYRO Y (deg/s)', 'LDelt_GyroZTime Series(s)', 'LDelt_GYRO Z (deg/s)',
                 'RBicep_TimeSeries', 'RBicep_MilliVolts', 'RBicep_Acc X Time Series(s)', 'RBicep_ACC X (G)', 'RBicep_Acc Y Time Series(s)',   'RBicep_ACC Y (G)',  'RBicep_Acc Z Time Series(s)', 'RBicep_ACC Z (G)','RBicep_GyroXTime Series(s)', 'RBicep_GYRO X (deg/s)','RBicep_GyroYTime Series(s)', 'RBicep_GYRO Y (deg/s)', 'RBicep_GyroZTime Series(s)', 'RBicep_GYRO Z (deg/s)',
                 'LBicep_TimeSeries', 'LBicep_MilliVolts', 'LBicep_Acc X Time Series(s)', 'LBicep_ACC X (G)', 'LBicep_Acc Y Time Series(s)',   'LBicep_ACC Y (G)',  'LBicep_Acc Z Time Series(s)', 'LBicep_ACC Z (G)','LBicep_GyroXTime Series(s)', 'LBicep_GYRO X (deg/s)','LBicep_GyroYTime Series(s)', 'LBicep_GYRO Y (deg/s)', 'LBicep_GyroZTime Series(s)', 'LBicep_GYRO Z (deg/s)'
                ]
    return df #raw data 

In [19]:
def column_clean(df, run_num, gender):
    #remove all time series columns except RDelt_EMG_TimeSeries' and 'RDelt_IMU_Acc X Time Series(s)', so keep time scale for both EMG and IMU 
    extr_time_series = [ 'RDelt_Acc Y Time Series(s)', 'RDelt_Acc Z Time Series(s)', 'RDelt_GyroXTime Series(s)',
                         'RDelt_GyroYTime Series(s)', 'RDelt_GyroZTime Series(s)', 'LDelt_TimeSeries', 'LDelt_Acc X Time Series(s)', 
                         'LDelt_Acc Y Time Series(s)', 'LDelt_Acc Z Time Series(s)', 'LDelt_GyroXTime Series(s)',
                         'LDelt_GyroYTime Series(s)', 'LDelt_GyroZTime Series(s)', 'RBicep_TimeSeries', 'RBicep_Acc X Time Series(s)',
                         'RBicep_Acc Y Time Series(s)', 'RBicep_Acc Z Time Series(s)', 'RBicep_GyroXTime Series(s)',
                         'RBicep_GyroYTime Series(s)', 'RBicep_GyroZTime Series(s)', 'LBicep_TimeSeries', 'LBicep_Acc X Time Series(s)',
                         'LBicep_Acc Y Time Series(s)', 'LBicep_Acc Z Time Series(s)', 'LBicep_GyroXTime Series(s)', 
                         'LBicep_GyroYTime Series(s)', 'LBicep_GyroZTime Series(s)']
    
    df = df.drop(extr_time_series, axis = 1)
    # measurement_cols = [col for col in df.columns if (('ACC' in col or 'GYRO' in col) and 'Time Series' not in col)] #exclude mV and time cols
    # df.columns = df.columns.str.strip()           # Remove leading/trailing spaces (Yuxuan)
    # df = df.apply(pd.to_numeric, errors='coerce') # Conver  t everything to numeric (Yuxuan)
    df = df.replace(['', ' ', 'NA', None], np.nan) #stdize missing data
    df['gender'] = gender
    df['run_num'] = run_num
    return df 

In [20]:
def create_sensor_col(df): 
    columns_to_keep = ['RDelt_EMG_TimeSeries', 'RDelt_IMU_Acc X Time Series(s)', 'gender', 'run_num']
    pdb.set_trace()
    # Identify all measurement columns, including EMG millivolts
    measurement_columns = [col for col in df.columns if any(prefix in col for prefix in ['RDelt', 'LDelt', 'RBicep', 'LBicep']) and col not in columns_to_keep]
    df_melted = df.melt(id_vars=columns_to_keep, value_vars=measurement_columns, var_name="sensor_measurement", value_name="value")
    
    # Extract the Sensor Body Position
    df_melted["Sensor_Body_Position"] = df_melted["sensor_measurement"].str.extract(r'^(RDelt|LDelt|RBicep|LBicep)')
    # Extract measurement type, including EMG millivolts
    df_melted["measurement_type"] = df_melted["sensor_measurement"].str.extract(r'_(EMG_MilliVolts|MilliVolts|ACC X|ACC Y|ACC Z|GYRO X|GYRO Y|GYRO Z)')    # Drop the original sensor_measurement column
    df_melted = df_melted.drop(columns=["sensor_measurement"])    # Pivot the DataFrame so each measurement type becomes a separate column
    df_melted["value"] = df_melted["value"].astype(str).str.strip() #make sure that values are clean if they're strings (no extra space)
    df_melted["value"] = pd.to_numeric(df_melted["value"], errors="coerce") #make sure all values are cast to numeric
    df_melted.fillna(np.nan, inplace=True)
    # Pivot the DataFrame so each measurement type becomes a separate column
    df_pivoted = df_melted.pivot_table(index=['Sensor_Body_Position', 'RDelt_EMG_TimeSeries', 'RDelt_IMU_Acc X Time Series(s)', 'gender', 'run_num'], 
                                    columns='measurement_type', values='value')
    df_pivoted.columns = df_pivoted.columns.get_level_values(0)
    df_pivoted.columns = df_pivoted.columns.str.strip()
    df_pivoted = df_pivoted.reset_index()
    pdb.set_trace()
    df_pivoted.to_csv("pivoted_df.csv")
    return df_pivoted

In [5]:
def standardize_time_series():
    # interpolate()
    pass

def preprocessing(full_df):
    pass

In [6]:
# Calculations for Feature Extraction from Project_Guide
def compute_emg_features(signal):
    return {
        'mean': np.mean(signal),
        'max': np.max(signal),
        'min': np.min(signal),
        'std': np.std(signal),
        'rms': np.sqrt(np.mean(signal**2))
    }

def compute_accel_features(a_x, a_y, a_z):
    a_mag = np.sqrt(a_x**2 + a_y**2 + a_z**2)
    
    features = {
        'peak_accel': np.max(a_mag),
        'mean_accel': np.mean(a_mag),
        'total_accel': np.sqrt(np.mean(a_x**2) + np.mean(a_y**2) + np.mean(a_z**2)),
        'accel_range': np.max(a_mag) - np.min(a_mag)
    }
    return features

def compute_gyro_features(w_x, w_y, w_z):
    w_mag = np.sqrt(w_x**2 + w_y**2 + w_z**2)
    
    features = {
        'peak_angular_vel': np.max(w_mag),
        'mean_angular_vel': np.mean(w_mag),
        'total_angular_vel': np.sqrt(np.mean(w_x**2) + np.mean(w_y**2) + np.mean(w_z**2)),
        'angular_vel_range': np.max(w_mag) - np.min(w_mag)
    }
    return features    

In [ ]:
P3exo_feats, p3noexo_feats, p4exo_feats, p4noexo_feats = overall_cleaning()

In [7]:
df_p3_exo = read_run("P3_Exo_1_0.csv") # 2nd run, male

In [8]:
df_p3_noexo = read_run("P3_NoExo_1_0.csv") # first run, male

In [9]:
df_p4_exo = read_run("P4_Exo_1_0.csv") # 1st run female
df_p4_noexo = read_run("P4_NoExo_1_0.csv") # 2nd female

In [10]:
df_p3_exo = column_clean(df_p3_exo, run_num = 2, gender = 'male')
df_p3_noexo = column_clean(df_p3_noexo, run_num = 1, gender = 'male')
df_p4_exo = column_clean(df_p4_exo, run_num = 2, gender = 'female')
df_p4_noexo = column_clean(df_p4_noexo, run_num = 1, gender = 'female')

In [16]:
df = df_p3_exo
columns_to_keep = ['RDelt_EMG_TimeSeries', 'RDelt_IMU_Acc X Time Series(s)', 'gender', 'run_num']
measurement_columns = [col for col in df.columns if any(prefix in col for prefix in ['RDelt', 'LDelt', 'RBicep', 'LBicep']) and col not in columns_to_keep]
df_melted = df.melt(id_vars=columns_to_keep, value_vars=measurement_columns, var_name="sensor_measurement", value_name="value")
    # Extract the Sensor Body Position
df_melted["Sensor_Body_Position"] = df_melted["sensor_measurement"].str.extract(r'^(RDelt|LDelt|RBicep|LBicep)')
# Extract measurement type, including EMG millivolts
df_melted["measurement_type"] = df_melted["sensor_measurement"].str.extract(r'_(EMG_MilliVolts|MilliVolts|ACC X|ACC Y|ACC Z|GYRO X|GYRO Y|GYRO Z)') 
df_melted

,RDelt_EMG_TimeSeries,RDelt_IMU_Acc X Time Series(s),gender,run_num,sensor_measurement,value,Sensor_Body_Position,measurement_type
0,0.000000,0,male,2,RDelt_EMG_MilliVolts,0.004868,RDelt,EMG_MilliVolts
1,0.000794,0.00675,male,2,RDelt_EMG_MilliVolts,0.005875,RDelt,EMG_MilliVolts
2,0.001588,0.0135,male,2,RDelt_EMG_MilliVolts,0.005203,RDelt,EMG_MilliVolts
3,0.002382,0.02025,male,2,RDelt_EMG_MilliVolts,0.005539,RDelt,EMG_MilliVolts
4,0.003177,0.027,male,2,RDelt_EMG_MilliVolts,0.007721,RDelt,EMG_MilliVolts
...,...,...,...,...,...,...,...,...
3852739,109.265029,NaN,male,2,LBicep_GYRO Z (deg/s),NaN,LBicep,GYRO Z
3852740,109.265823,NaN,male,2,LBicep_GYRO Z (deg/s),NaN,LBicep,GYRO Z
3852741,109.266618,NaN,male,2,LBicep_GYRO Z (deg/s),NaN,LBicep,GYRO Z
3852742,109.267412,NaN,male,2,LBicep_GYRO Z (deg/s),NaN,LBicep,GYRO Z


In [ ]:
    # Extract the Sensor Body Position
df_melted["Sensor_Body_Position"] = df_melted["sensor_measurement"].str.extract(r'^(RDelt|LDelt|RBicep|LBicep)')
# Extract measurement type, including EMG millivolts
df_melted["measurement_type"] = df_melted["sensor_measurement"].str.extract(r'_(EMG_MilliVolts|MilliVolts|ACC X|ACC Y|ACC Z|GYRO X|GYRO Y|GYRO Z)')    # Drop the original sensor_measurement column
df_melted = df_melted.drop(columns=["sensor_measurement"])    # Pivot the DataFrame so each measurement type becomes a separate column
df_melted["value"] = df_melted["value"].astype(str).str.strip() #make sure that values are clean if they're strings (no extra space)
df_melted["value"] = pd.to_numeric(df_melted["value"], errors="coerce") #make sure all values are cast to numeric
df_melted.fillna(np.nan, inplace=True)
# Pivot the DataFrame so each measurement type becomes a separate column
df_pivoted = df_melted.pivot_table(index=['Sensor_Body_Position', 'RDelt_EMG_TimeSeries', 'RDelt_IMU_Acc X Time Series(s)', 'gender', 'run_num'], 
                                columns='measurement_type', values='value')
df_pivoted.columns = df_pivoted.columns.get_level_values(0)
df_pivoted.columns = df_pivoted.columns.str.strip()
df_pivoted = df_pivoted.reset_index()

,RDelt_EMG_TimeSeries,RDelt_EMG_MilliVolts,RDelt_IMU_Acc X Time Series(s),RDelt_ACC X (G),RDelt_ACC Y (G),RDelt_ACC Z (G),RDelt_GYRO X (deg/s),RDelt_GYRO Y (deg/s),RDelt_GYRO Z (deg/s),LDelt_MilliVolts,...,RBicep_GYRO Z (deg/s),LBicep_MilliVolts,LBicep_ACC X (G),LBicep_ACC Y (G),LBicep_ACC Z (G),LBicep_GYRO X (deg/s),LBicep_GYRO Y (deg/s),LBicep_GYRO Z (deg/s),gender,run_num
0,0.000000,0.004868,0,0.0747681,0.9061279,0.2548828,-30.7404575,-4.2519083,9.358779,-0.006546,...,-11.9541988,0.041962,0.2507324,0.8808594,0.1972656,-30.801527,8.5572519,12.6870232,male,2
1,0.000794,0.005875,0.00675,0.0795288,0.913208,0.2689209,-30.7786255,-5.961832,8.5419846,-0.006546,...,-12.5343513,0.041962,0.2453003,0.8790283,0.2055054,-29.038168,9.9007635,13.0305347,male,2
2,0.001588,0.005203,0.0135,0.0804443,0.9194336,0.2719116,-29.9312973,-6.8015265,8.4503813,-0.007217,...,-12.801527,0.041459,0.2486572,0.880188,0.2092896,-27.6641216,8.9694653,12.358779,male,2
3,0.002382,0.005539,0.02025,0.0809326,0.9316406,0.2680054,-29.1068707,-6.8854961,7.6793895,-0.004196,...,-12.7480917,0.039780,0.2533569,0.880127,0.2134399,-25.442749,7.6106873,11.1145039,male,2
4,0.003177,0.007721,0.027,0.0866699,0.9319458,0.2663574,-29.3129768,-7.4045801,6.7557254,-0.005203,...,-11.1832066,0.041459,0.2590942,0.8770752,0.2124634,-23.557251,6.0916033,9.7862597,male,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
137593,109.265029,0.017960,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.002182,...,NaN,0.030716,NaN,NaN,NaN,NaN,NaN,NaN,male,2
137594,109.265823,0.019974,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.001846,...,NaN,0.035416,NaN,NaN,NaN,NaN,NaN,NaN,male,2
137595,109.266618,0.020981,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.004196,...,NaN,0.034745,NaN,NaN,NaN,NaN,NaN,NaN,male,2
137596,109.267412,0.018631,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.005707,...,NaN,0.035248,NaN,NaN,NaN,NaN,NaN,NaN,male,2


In [ ]:
df_p3_exo = create_sensor_col(df_p3_exo)
df_p3_noexo = create_sensor_col(df_p3_noexo)
df_p4_exo = create_sensor_col(df_p4_exo)
df_p4_noexo = create_sensor_col(df_p4_noexo)

In [ ]:
dfs = [df_p3_exo, df_p3_noexo, df_p4_exo, df_p4_noexo] #jack's list for the data cleaning he does later.
combined_df = pd.concat(dfs, ignore_index=True)

In [ ]:
# # Show the head of the data
# df_p3_exo.describe()
df_p3_noexo.head()
# df_p4_exo.head()
# df_p4_noexo.head()
# # Choose inputs
# features = df_p3_exo[['EMG 1 (mV)', 'ACC X (G)', 'ACC Y (G)', 'ACC Z (G)', 'GYRO X (deg/s)', 'GYRO Y (deg/s)', 'GYRO Z (deg/s)']].dropna()
# features.head()
feature_sets = []
# Run functions to extract features for each dataframe
#CP: does this make sure to remove the redundant time series columns?
#can keep  ACC X Time Series (s) in each sensor group, and remove any other column with 'Time Series (s)' in its name 
for df in dfs:
    emg_features = compute_emg_features(df['EMG 1 (mV)'])
    accel_features = compute_accel_features(df['ACC X (G)'], df['ACC Y (G)'], df['ACC Z (G)'])
    gyro_features = compute_gyro_features(df['GYRO X (deg/s)'], df['GYRO Y (deg/s)'], df['GYRO Z (deg/s)'])
    features = {
        'emg': emg_features,
        'accel': accel_features,
        'gyro': gyro_features
    }
    feature_sets.append(features)